In [1]:
import numpy as np 

In [139]:
class ScratchLogisticRegression():
    """
    Scratch implementation of logistic regression

    Parameters
    ----------
    num_iter : int
      Number of iterations
    lr : float
      Learning rate
    no_bias : bool
      True if no bias term is included
    verbose : bool
      True to output the learning process

    Attributes
    ----------
    self.coef_ : The following form of ndarray, shape (n_features,)
      Parameters
    self.loss : The following form of ndarray, shape (self.iter,)
      Record losses on training data
    self.val_loss : The following form of ndarray, shape (self.iter,)
      Record loss on validation data

    """

    def __init__(self, num_iter = 100, lr = 1e-5, bias=False, verbose=False, lamb=0.1):
        # Record hyperparameters as attributes
        self.iter = num_iter
        self.lr = lr
        self.bias = bias
        self.verbose = verbose
        # Prepare an array to record the loss
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        # parameter
        self.theta = np.array([])
        self.lamb = lamb
        
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        Learn logistic regression. If validation data is entered, the loss and accuracy for it are also calculated for each iteration.

        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            Features of training data
        y : The following form of ndarray, shape (n_samples,)
            Correct answer value of training data
        X_val : The following forms of ndarray, shape (n_samples, n_features)
            Features of verification data
        y_val : The following form of ndarray, shape (n_samples,)
            Correct value of verification data
        """
        if self.bias == True:
            a = np.ones(X.shape[0]).reshape((-1, 1))
            X = np.hstack([a, X])

        self.theta = np.zeros(X.shape[1]).reshape((-1, 1))
        for i in range(self.iter):
            self._gradient_descent(X, y)
            y_pred = self._logistic_hypothesis(X)
            self.loss = np.append(self.loss, self._loss_func(y_pred, y))
        return self

    def predict(self, X):
        """
        Estimate the label using logistic regression.

        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            sample

        Returns
        -------
            The following form of ndarray, shape (n_samples, 1)
            Estimated result by logistic regression
        """
        if self.bias == True:
            a = np.ones(X.shape[0]).reshape((-1, 1))
            X = np.hstack([a, X])


        return np.where(self._logistic_hypothesis(X) >=0.5, 1, 0)

       

    def predict_proba(self, X):
        """
        Estimate the probability using logistic regression.

        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            sample

        Returns
        -------
            The following form of ndarray, shape (n_samples, 1)
            Estimated result by logistic regression
        """
        if self.bias == True:
            a = np.ones(X.shape[0]).reshape((-1, 1))
            X = np.hstack([a, X])
        return self._logistic_hypothesis(X)
    
    def hypothesis(self, x):
        return x @ self.theta
    
    def _logistic_hypothesis(self, x):
        y_pred = self.hypothesis(x)
        return 1/ (1 + np.exp(-y_pred))
    
    # gradient descent is buggy
    def _gradient_descent(self, x, y):
        rows = x.shape[0]
        columns = x.shape[1]
        y_pred = self._logistic_hypothesis(x)

        for column in range(columns):

            gradient = 0
            for row in range(rows):
                gradient += (y_pred[row] - y[row]) * x[row, columns]
            self.theta[column] = self.theta[column] - self.lr * ( gradient + self.lamb*self.theta[column])/rows
            
    
    def _loss_func(self, pred, y):
        error = 0
        for i in range(y.shape[0]):
            error += -np.sum(y[i] *  np.log(pred[i])+(1-y[i]) *  np.log(1-pred[i]))
        loss = error / (y.shape[0])
        loss = loss + np.sum(self.theta**2)*self.lamb/(2 * y.shape[0])
        return loss

        


  

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


iris = load_iris()

features = iris.data
target = iris.target 

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [121]:
def sigmoid(X):
    return 1 / (1 + np.exp(X))



from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(x_train, y_train)

model.coef_

array([[ 0.51526936, -0.78879272,  2.16216075,  0.90876014]])

In [122]:
a = np.array([-0.50863213,  0.75978407, -2.38787951, -0.94154691]).reshape((-1, 1))


In [123]:
sigmoid(x_test @ a)

array([[0.98404322],
       [0.99999801],
       [0.99999958],
       [0.96892367],
       [0.99992443],
       [0.98624802],
       [0.98458867],
       [0.976894  ],
       [0.98620355],
       [0.99993343],
       [0.99999891],
       [0.99999605],
       [0.99024753],
       [0.9760031 ],
       [0.97264521],
       [0.98361745],
       [0.96865292],
       [0.95095343],
       [0.98029721],
       [0.99999701]])

In [124]:
model.predict(x_test)

array([0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [125]:
from sklearn.datasets import load_iris
import pandas as pd 
iris = load_iris(as_frame=True)

data = pd.concat([iris.data, iris.target],axis=1)

data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [126]:
data = data[data["target"] != 2]

x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1] , data.iloc[:, -1],test_size=0.2)
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

In [140]:
model = ScratchLogisticRegression(lr=1e-3).fit(x_train, y_train)

In [142]:
a = model.theta

In [144]:
sigmoid(x_test @ a)


array([[0.45263746],
       [0.5010632 ],
       [0.50046433],
       [0.44644599],
       [0.44864311],
       [0.44446812],
       [0.50851554],
       [0.44692253],
       [0.4520566 ],
       [0.44717246],
       [0.45737193],
       [0.45376713],
       [0.49949356],
       [0.44663794],
       [0.50696824],
       [0.44729718],
       [0.49809433],
       [0.50014386],
       [0.50497188],
       [0.45656845]])